In [ ]:
#!pip3 install imageai --upgrade

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

print(os.listdir("./FSC/aa_final/train/"))
# Any results you write to the current directory are saved as output.

In [ ]:
DATA_PATH = "./FSC/aa_final/train/"


In [ ]:
PRETRAINED_MODEL = None

In [ ]:
import h5py
import torch
import shutil

def save_net(fname, net):
    with h5py.File(fname, 'w') as h5f:
        for k, v in net.state_dict().items():
            h5f.create_dataset(k, data=v.cpu().numpy())
def load_net(fname, net):
    with h5py.File(fname, 'r') as h5f:
        for k, v in net.state_dict().items():        
            param = torch.from_numpy(np.asarray(h5f[k]))         
            v.copy_(param)
            
def save_checkpoint(state, is_best,task_id, filename='checkpoint.pth.tar'):
    torch.save(state, task_id+filename)
    if is_best:
        shutil.copyfile(task_id+filename, task_id+'model_best.pth.tar')   

In [ ]:
"""
contain dummy args with config
helpfull for copy paste Kaggle
"""
import argparse


def make_args(train_json= "", test_json="", pre="", gpu="0", task="task_one_"):
    """
    these arg does not have any required commandline arg (all with default value)
    :param train_json:
    :param test_json:
    :param pre:
    :param gpu:
    :param task:
    :return:
    """
    parser = argparse.ArgumentParser(description='PyTorch CSRNet')

    args = parser.parse_args()
    args.gpu = gpu
    args.task = task
    args.pre = None
    return args


class Meow():
    def __init__(self):
        pass


def make_meow_args(gpu="0", task="task_one_"):
    args = Meow()
    args.gpu = gpu
    args.task = task
    args.pre = None
    return args

In [ ]:
import os
import glob
from sklearn.model_selection import train_test_split
import json
"""
create a list of file (full directory)
"""

def create_training_image_list(data_path):
    """
    create a list of absolutely path of jpg file
    :param data_path: must contain subfolder "images" with *.jpg  (example ShanghaiTech/QNRF/train_data/)
    :return:
    """
    DATA_PATH = data_path
    image_path_list = glob.glob(os.path.join(DATA_PATH, "images", "*.jpg"))
    return image_path_list

def get_train_val_list(data_path):
    DATA_PATH = data_path
    image_path_list = glob.glob(os.path.join(DATA_PATH, "images", "*.jpg"))
    train, val = train_test_split(image_path_list, test_size=0.2, random_state=113)

    print("train size ", len(train))
    print("val size ", len(val))
    return train, val

In [ ]:
import random
import os
from PIL import Image,ImageFilter,ImageDraw
import numpy as np
import h5py
from PIL import ImageStat
import cv2

def load_data(img_path,train = True):
    gt_path = img_path.replace('.jpg','.h5').replace('images','ground-truth')
    img = Image.open(img_path).convert('RGB')
    gt_file = h5py.File(gt_path, 'r')
    target = np.asarray(gt_file['density'])

    target = cv2.resize(target,(int(target.shape[1]/8), int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64
    
    return img,target

In [ ]:
import os
import random
import torch
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms.functional as F
class ListDataset(Dataset):
    def __init__(self, root, shape=None, shuffle=True, transform=None,  train=False, seen=0, batch_size=1, num_workers=4):
        """
        if you have different image size, then batch_size must be 1
        :param root:
        :param shape:
        :param shuffle:
        :param transform:
        :param train:
        :param seen:
        :param batch_size:
        :param num_workers:
        """
        if train:
            root = root *2
        if shuffle:
            random.shuffle(root)
        
        self.nSamples = len(root)
        self.lines = root
        self.transform = transform
        self.train = train
        self.shape = shape
        self.seen = seen
        self.batch_size = batch_size
        self.num_workers = num_workers
        
    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        assert index <= len(self), 'index range error' 
        
        img_path = self.lines[index]
        
        img,target = load_data(img_path,self.train)
        
        #img = 255.0 * F.to_tensor(img)
        
        #img[0,:,:]=img[0,:,:]-92.8207477031
        #img[1,:,:]=img[1,:,:]-95.2757037428
        #img[2,:,:]=img[2,:,:]-104.877445883


        
        
        if self.transform is not None:
            img = self.transform(img)
        return img,target
        

In [ ]:
import h5py
import torch
import shutil

def save_net(fname, net):
    with h5py.File(fname, 'w') as h5f:
        for k, v in net.state_dict().items():
            h5f.create_dataset(k, data=v.cpu().numpy())
def load_net(fname, net):
    with h5py.File('./'+fname, 'r') as h5f:
        for k, v in net.state_dict().items():        
            param = torch.from_numpy(np.asarray(h5f[k]))         
            v.copy_(param)
            
def save_checkpoint(state, is_best,task_id, filename='checkpoint.pth.tar'):
    torch.save(state, './'+task_id+filename)
    if is_best:
        shutil.copyfile('./'+task_id+filename, './'+task_id+'model_best.pth.tar')            

def tv_loss(y):
    loss = torch.sum((y[:-1, :, :, :] - y[1:, :, :, :])**2) + torch.sum((y[:, :-1, :, :] - y[:, 1:, :, :])**2)
    return loss


In [ ]:
"""
Student model (1/n-CSRNet) in SKT.
"""
import torch.nn as nn
import torch
from torchvision import models

channel_nums = [[32, 64, 128, 256],  # half
                [21, 43, 85, 171],  # third
                [16, 32, 64, 128],  # quarter
                [13, 26, 51, 102],  # fifth
                ]


class SKT(nn.Module):
    def __init__(self, ratio=4, transform=True):
        super(SKT, self).__init__()
        self.seen = 0
        self.transform = transform
        channel = channel_nums[ratio-2]
        self.conv0_0 = conv_layers(3, channel[0])
        if self.transform:
            self.transform0_0 = feature_transform(channel[0], 64)
        self.conv0_1 = conv_layers(channel[0], channel[0])

        self.pool0 = pool_layers()
        if transform:
            self.transform1_0 = feature_transform(channel[0], 64)
        self.conv1_0 = conv_layers(channel[0], channel[1])
        self.conv1_1 = conv_layers(channel[1], channel[1])

        self.pool1 = pool_layers()
        if transform:
            self.transform2_0 = feature_transform(channel[1], 128)
        self.conv2_0 = conv_layers(channel[1], channel[2])
        self.conv2_1 = conv_layers(channel[2], channel[2])
        self.conv2_2 = conv_layers(channel[2], channel[2])

        self.pool2 = pool_layers()
        if transform:
            self.transform3_0 = feature_transform(channel[2], 256)
        self.conv3_0 = conv_layers(channel[2], channel[3])
        self.conv3_1 = conv_layers(channel[3], channel[3])
        self.conv3_2 = conv_layers(channel[3], channel[3])

        self.conv4_0 = conv_layers(channel[3], channel[3], dilation=2)
        #if transform:
        #    self.transform4_0 = feature_transform(channel[3], 512)
        #self.conv4_1 = conv_layers(channel[3], channel[3], dilation=2)
        #self.conv4_2 = conv_layers(channel[3], channel[3], dilation=2)
        #self.conv4_3 = conv_layers(channel[3], channel[2], dilation=2)
        #if transform:
        #    self.transform4_3 = feature_transform(channel[2], 256)
        #self.conv4_4 = conv_layers(channel[2], channel[1], dilation=2)
        #self.conv4_5 = conv_layers(channel[1], channel[0], dilation=2)

        #self.conv5_0 = nn.Conv2d(channel[0], 1, kernel_size=1)

        self._initialize_weights()
        self.features = []

    def forward(self, x):
        self.features = []

        x = self.conv0_0(x)
        if self.transform:
            self.features.append(self.transform0_0(x))
        x = self.conv0_1(x)

        x = self.pool0(x)
        if self.transform:
            self.features.append(self.transform1_0(x))
        x = self.conv1_0(x)
        x = self.conv1_1(x)

        x = self.pool1(x)
        if self.transform:
            self.features.append(self.transform2_0(x))
        x = self.conv2_0(x)
        x = self.conv2_1(x)
        x = self.conv2_2(x)

        x = self.pool2(x)
        if self.transform:
            self.features.append(self.transform3_0(x))
        x = self.conv3_0(x)
        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.conv4_0(x)
        #if self.transform:
        #    self.features.append(self.transform4_0(x))
        #x = self.conv4_1(x)
        #x = self.conv4_2(x)
        #x = self.conv4_3(x)
        #if self.transform:
        #    self.features.append(self.transform4_3(x))
        #x = self.conv4_4(x)
        #x = self.conv4_5(x)

        #x = self.conv5_0(x)

        #self.features.append(x)
        #print('x3 :', x.size())
        #x = torch.nn.Upsample(size=(67, 120), mode='bilinear')(x)
        #print('x3 :', x.size())
        #if self.training is True:
        #    return self.features
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # nn.init.xavier_normal_(m.weight)
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                # nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


def conv_layers(inp, oup, dilation=False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    return nn.Sequential(
        nn.Conv2d(inp, oup, kernel_size=3, padding=d_rate, dilation=d_rate),
        nn.ReLU(inplace=True)
    )


def feature_transform(inp, oup):
    conv2d = nn.Conv2d(inp, oup, kernel_size=1)  # no padding
    relu = nn.ReLU(inplace=True)
    layers = []
    layers += [conv2d, relu]
    return nn.Sequential(*layers)


def pool_layers(ceil_mode=True):
    return nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=ceil_mode)

        



In [ ]:
'''
spn = SKT()
print(spn)

model = SKT()
x = torch.randn(1, 3, 768, 1024)

# Let's print it
model(x)
'''

In [ ]:
import sys
import os
import math
import warnings

# import from library
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import argparse
import json
import cv2
import time

"""
A dublicate of train.py 
However it does not need commandline arg
"""
def main():
    global args, best_prec1
    args = make_meow_args()


    best_prec1 = 1e6

    args.original_lr = 1e-7
    args.lr = 1e-7
    args.batch_size = 1
    args.momentum = 0.95
    args.decay = 5 * 1e-4
    args.start_epoch = 0
    args.epochs = 50
    args.steps = [-1, 1, 100, 150]
    args.scales = [1, 1, 1, 1]
    args.workers = 4
    args.seed = time.time()
    args.print_freq = 30
    args.pre = PRETRAINED_MODEL

    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    torch.cuda.manual_seed(args.seed)

    train_list, val_list = get_train_val_list(DATA_PATH)

    model = SKT()

    model = model.cuda()

    criterion = nn.MSELoss(size_average=False).cuda()

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.decay)

    if args.pre:
        if os.path.isfile(args.pre):
            print("=> loading checkpoint '{}'".format(args.pre))
            checkpoint = torch.load(args.pre)
            args.start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.pre, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.pre))

    for epoch in range(args.start_epoch, args.epochs):
        adjust_learning_rate(optimizer, epoch)

        train(train_list, model, criterion, optimizer, epoch)
        prec1 = validate(val_list, model, criterion)
        is_best = prec1 < best_prec1
        best_prec1 = min(prec1, best_prec1)
        print(' * best MAE {mae:.3f} '
              .format(mae=best_prec1))
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': args.pre,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, args.task)


def train(train_list, model, criterion, optimizer, epoch):
    losses = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    train_loader = torch.utils.data.DataLoader(
        ListDataset(train_list,
                            shuffle=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                            std=[0.229, 0.224, 0.225]),
                            ]),
                            train=True,
                            seen=model.seen,
                            batch_size=args.batch_size,
                            num_workers=args.workers),
        batch_size=args.batch_size)
    print('epoch %d, processed %d samples, lr %.10f' % (epoch, epoch * len(train_loader.dataset), args.lr))

    model.train()
    end = time.time()

    for i, (img, target) in enumerate(train_loader):
        data_time.update(time.time() - end)

        img = img.cuda()
        img = Variable(img)
        output = model(img)

        target = target.type(torch.FloatTensor).unsqueeze(0).cuda()
        target = Variable(target)
        
        #print('out ', output.size(),' tar ',target.size())
        #target.reshape([1, 1, 34, 120]) #out[1, 1, 34, 120])#tar[1, 1, 67, 120]
        loss = criterion(output, target)

        losses.update(loss.item(), img.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                .format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses))


def validate(val_list, model, criterion):
    print('begin test')
    test_loader = torch.utils.data.DataLoader(
        ListDataset(val_list,
                            shuffle=False,
                            transform=transforms.Compose([
                                transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                            std=[0.229, 0.224, 0.225]),
                            ]), train=False),
        batch_size=args.batch_size)

    model.eval()

    mae = 0
    mse = 0
    for i, (img, target) in enumerate(test_loader):
        img = img.cuda()
        img = Variable(img)
        output = model(img)

        mae += abs(output.data.sum() - target.sum().type(torch.FloatTensor).cuda())
        mse += abs(output.data.sum() - target.sum().type(torch.FloatTensor).cuda())*abs(output.data.sum() - target.sum().type(torch.FloatTensor).cuda())


    mae = mae / len(test_loader)
    mse = math.sqrt(mse/len(test_loader))
    print(' * MSE {mse:.3f} '.format(mse=mse))
    print(' * MAE {mae:.3f} '
          .format(mae=mae))

    return mae

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""

    args.lr = args.original_lr

    for i in range(len(args.steps)):

        scale = args.scales[i] if i < len(args.scales) else 1

        if epoch >= args.steps[i]:
            args.lr = args.lr * scale
            if epoch == args.steps[i]:
                break
        else:
            break
    for param_group in optimizer.param_groups:
        param_group['lr'] = args.lr

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
if __name__ == '__main__':

    main()

In [ ]:
#test_image_list = create_training_image_list(TEST_DATA_PATH)


In [ ]:
#train_list, val_list = get_train_val_list(DATA_PATH)
#test_loader = torch.utils.data.DataLoader(
#        ListDataset(val_list,
#                            shuffle=False,
#                            transform=transforms.Compose([
#                                transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                                                            std=[0.229, 0.224, 0.225]),
#                            ]), train=False),
#        batch_size=args.batch_size)

In [ ]:
#print(test_loader)
#for i, (img, target) in enumerate(test_loader):
        #img = img.cuda()
        #img = Variable(img)
        #output = model(img)

        #mae += abs(output.data.sum() - target.sum().type(torch.FloatTensor).cuda())
train_list, val_list = get_train_val_list(DATA_PATH)
print('kkkk',len(val_list))
print('kkkk',os.path.splitext(os.path.basename(val_list[1]))[0])


In [ ]:
import glob
# importing libraries
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
from matplotlib import cm as CM
import torch
from tqdm import tqdm
%matplotlib inline

from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])
model = DenseScaleNet()
model = model.cuda()
checkpoint = torch.load('./task_one_model_best.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
#img_paths = []
#for path in path_sets:
#    print(path)
#    for img_path in glob.glob(os.path.join(path, '*.jpg')):
#        img_paths.append(img_path)
#        print(img_path)

In [ ]:


from matplotlib import cm as c
for i in range(len(val_list)):
    print('frame:',i)
    #print('kkkk',os.path.splitext(os.path.basename(val_list[i]))[0])
    ll=os.path.splitext(os.path.basename(val_list[i]))[0]
    img = transform(Image.open(val_list[i]).convert('RGB')).cuda()
    output = model(img.unsqueeze(0))
    #print("Predicted Count : ",int(output.detach().cpu().sum().numpy()))
    temp = np.asarray(output.detach().cpu().reshape(output.detach().cpu().shape[2],output.detach().cpu().shape[3]))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(int(output.detach().cpu().sum().numpy()))
    plt.imshow(temp,cmap = c.jet)
    plt.savefig('./data_results/DenseScaleNet_50_QNRF_'+ ll + '.png')
    plt.show()
    temp = h5py.File('QNRF/ground-truth/' + ll + '.h5', 'r')
    temp_1 = np.asarray(temp['density'])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(int(np.sum(temp_1)) + 1)
    plt.imshow(temp_1,cmap = c.jet)
    plt.savefig('./data_results/'+ ll + '_GT.png')
    #print("Original Count : ",int(np.sum(temp_1)) + 1)

In [ ]:
#from google.colab import files
#!ls

from google.colab import files
files.download('./task_one_model_best.pth.tar')
files.download('./task_one_checkpoint.pth.tar')
#torch.save('task_one_model_best.pth.tar', output_image_path)

In [ ]:
from google.colab import files
!ls
#!rm -rf Finale